In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing 
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#¿Las inversiones deben ser reportadas en el anexo es-4?
print('Ingrese la pregunta:')
pregunta = input()

Ingrese la pregunta:
Las inversiones deben ser reportadas en el anexo es-4


In [3]:
cadpun = string.punctuation + '¿�'
stops = stopwords.words('spanish')


pregunta = pregunta.lower()
pregunta = pregunta.translate(str.maketrans('', '', cadpun))
palabras = pregunta.split()
resultwords = [palabra for palabra in palabras if palabra.lower() not in stops]
preguntaclean = (' '.join(resultwords)).split()
preguntaclean

['inversiones', 'deben', 'ser', 'reportadas', 'anexo', 'es4']

In [4]:
tagged_question = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(pregunta)]
palabras = pregunta.split()

In [5]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape


(1301830, 300)

In [6]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
documentos = bdtesis.documentosSBS


In [8]:
vector = model.infer_vector(preguntaclean).reshape(1,-1)
vectores = []
documentos = []
for x in bdtesis.documentosSBS.find():
    vectores.append(x['vector'])
for x in bdtesis.documentosSBS.find():
    documentos.append(x['contenido'])

In [9]:
#comparación la pregunta ingresada con la BD
proximidades = []
maxprox = 0
cont = 0
similares = {}

for x in vectores:
    vectordoc = np.asarray(x).reshape(1,-1)
    res = cosine_similarity(vectordoc,vector) 
    #similares[float(res[0])] = preguntas[cont]
    similares[documentos[cont]] = float(res[0]) 
    if (res> maxprox):
        docaprox = documentos[cont]
        maxprox = res
        contprox = cont
    proximidades.append(res)
    cont = cont + 1
docaprox


'El cálculo del indicador de medición del riesgo cambiario deberá ser incluido en el Anexo ES-19A.'

In [10]:
max(proximidades)

array([[0.60707575]])

In [11]:
ordenados = sorted(similares.items(), key=lambda kv: kv[1], reverse=True)


In [12]:
print(ordenados[0])
print(ordenados[1])
print(ordenados[2])
print(ordenados[3])
print(ordenados[4])


('El cálculo del indicador de medición del riesgo cambiario deberá ser incluido en el Anexo ES-19A.', 0.6070757524087512)
('Los instrumentos emitidos en el exterior, vía oferta pública o privada, deben ser clasificados, por lo menos, por una (1) empresa clasificadora señalada en el Anexo N° 1 del presente Reglamento. ', 0.5937664502248429)
('El Anexo N° ES-4 Inversiones totales se remite con periodicidad mensual ', 0.5900275902807837)
('Contar con indicadores de alerta temprana para identificar riesgos que afecten las inversiones en inmuebles u otras inversiones inmobiliarias. ', 0.5854306899936168)
('En el caso de instrumentos emitidos localmente vía oferta privada, las clasificaciones de riesgo pueden ser otorgadas por alguna de las empresas clasificadoras de riesgo señaladas en los literales a) o b); y se considera como grado de inversión lo indicado en el literal que corresponda, según la equivalencia establecida en el Anexo antes citado. ', 0.5823252738518438)
